<a href="https://colab.research.google.com/github/kn0wthing/practice/blob/main/Practice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
kaggle_dir = '/root/.kaggle'
os.makedirs(kaggle_dir, exist_ok=True)


In [5]:
!pwd

/content


In [9]:
!cp /content/drive/MyDrive/kaggle.json {kaggle_dir}/

In [10]:
!chmod 600 {kaggle_dir}/kaggle.json

In [17]:
!kaggle datasets download -d uciml/mushroom-classification

Dataset URL: https://www.kaggle.com/datasets/uciml/mushroom-classification
License(s): CC0-1.0
  0% 0.00/34.2k [00:00<?, ?B/s]
100% 34.2k/34.2k [00:00<00:00, 41.7MB/s]


In [18]:
!unzip /content/mushroom-classification.zip

Archive:  /content/mushroom-classification.zip
  inflating: mushrooms.csv           


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [20]:
df = pd.read_csv('/content/mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [23]:
X, y = df.iloc[:, 1:], df.iloc[:, 0]

In [29]:
le = LabelEncoder()
for col in X.columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
y = le.fit_transform(y)

<ipython-input-29-bd06bf16bf33>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-29-bd06bf16bf33>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-29-bd06bf16bf33>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
class KNN(nn.Module):
    def __init__(self, k=5):
        self.k = k

    def _euclidean_distance(self, X1, X2):
        return torch.sqrt(((X1 - X2) ** 2).sum(dim=1))

    def _choice(self, neighbor_labels):
        counts = torch.bincount(neighbor_labels)
        return counts.argmax().item()

    def predict(self, X_test, X_train, y_train):
        y_pred = torch.empty(X_test.size(0), dtype=torch.long)
        for i, test_sample in enumerate(X_test):
            distances = self._euclidean_distance(test_sample, X_train)
            _, idx = distances.topk(self.k, largest=False)
            k_nearest_neighbors = y_train[idx]
            y_pred[i] = self._choice(k_nearest_neighbors)

        return y_pred


In [34]:
KNN_model = KNN(k=5)
y_pred = KNN_model.predict(torch.tensor(X_test.values, dtype=torch.float32), torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))

tensor([0, 1, 1,  ..., 1, 1, 1])